# tf-idf

We will get to machine learning soon. But in many cases, just counting words can tell you a lot. To wit:

<img src="http://lklein.com/wp-content/uploads/2021/10/Screen-Shot-2021-10-06-at-3.33.34-PM.png">

Today, we're going to explore a method called Term Frequency - Inverse Document Frequency (tf-idf). Tf-idf comes up a lot in text analysis projects because it’s both a corpus exploration method and a pre-processing step for many other text-mining measures and models.

The procedure was introduced in a 1972 paper by Karen Spärck Jones under the name “term specificity,” and the basic idea is this:

Instead of representing a term in a document by its raw frequency or its relative frequency (the term count divided by the document length), each term is *weighted* by dividing the term frequency by the number of documents in the corpus containing the word. 

The overall effect of this weighting scheme is to avoid a common problem when conducting text analysis: the most frequently used words in any particular document are often the most frequently used words in all of the documents.

By contrast, terms with the highest tf-idf scores are the terms in a document that are distinctively frequent in a document when that document is compared other documents. When you sort by tf-idf score, these distinctive terms rise to the top. 

## An Analogy ##
    
If this explanation doesn’t quite resonate, a brief analogy might help. 

Say you've decided leave campus to get dinner on Buford Highway. Since leaving campus takes a lot of effort (and also, crucially, access to a car), the food better be worth it! That means you'll need to balance two competing goals:

1) The food has to be really tasty; and also, crucially: 
2) If you're going to go all the way out to Buford Highway, it better be something that you can't also get in Emory Village. Otherwise, why go to all the trouble of getting there?!

Or, to give an example involving actual food: you don't want to go all the way out to Buford Highway to get pizza. Even if the pizza on Buford Highway is pretty tasty, you can get pizza anywhere in town. How can you find out what is distintively tasty on Buford Highway?  

If you looked up the Yelp reviews for the all restaurants on Buford highway and sorted by score, you would get an answer to the question of what's the tastiest. But it still won't help solve the problem of what's *distintively tasty* on Buford Highway--like hot pot, for example, which is something that you can't get in Emory Village.   

So you need a way to tell the difference between what's tasty and what's distinctively tasty. To do so, you need to distinguish between four categories of food. Food that, on Buford Highway, is:

- both tasty and distinctive (e.g. hot pot)
- tasty but not distinctive (e.g. pizza) 
- distinctive but not tasty (e.g. tacos-- tho I'm open to disagreement here)
- neither tasty nor distinctive (e.g. Taco Bell--again, open to disagreement).

These categories are what tf-idf helps you measure. Term frequencies can be assessed according to the same criteria. A term might be:

- Frequently used in the corpus, and used especially frequently in one particular document <-- Interesting! 
- Frequently used in the corpus, but used frequently in equal measure across all documents <-- Less interesting
- Infrequently used in the corpus, but nonetheless used frequently in one particular document <-- Potentially interesting
- Infrequently used in in the corpus and also infrequently used in the corpus consitently across all documents <-- Not interesting

It's the words that are especially frequent in one document that are most interesting to us, and the ones that TF/IDF helps us identify. To see how, let's take a look at our next corpus--a slightly bigger one--articles published in the *Emory Wheel*.

## *The Emory Wheel* 

In this lesson, we're going to use tf-idf to study the articles published by *The Emory Wheel* betweeen 2014 and 2019. This dataset was created by Honggang Min and Kexin Guan for their final project in the 2019 iteration of this course, and was generously transfered back to me for future class use.  

## Pre-processing: prepare the documents

Tf-idf works on sets of documents. We'll be using another (new to us) library, scikit-learn, in order to count the words. But before we do, we'll need to get the documents into a list, with each document stored as its own string. 

In this particular case, the documents are individual .txt files that are stored in a zip file on my Google Drive. Below is some code to get the data from Google Drive, unzipped, and formatted into a list for processing. 

Note that while this is custom code written for this particular dataset, you'll usually need to write some sort of file/text pre-processing code in order to use any particular library/method/tool. You'll get very familiar with writing code like this by the end of the course! 

In [1]:
# For downloading large files from Google Drive
# https://github.com/wkentaro/gdown
import gdown

# then download the zip file
gdown.download('https://drive.google.com/uc?export=download&id=1SUWUVswaY_RDLhzFruQIDJe-i6I3gznC', quiet=False) 

Downloading...
From: https://drive.google.com/uc?export=download&id=1SUWUVswaY_RDLhzFruQIDJe-i6I3gznC
To: /content/wheel-clean.zip
100%|██████████| 9.36M/9.36M [00:00<00:00, 22.2MB/s]


'wheel-clean.zip'

In [2]:
# unzip it 
!unzip wheel-clean.zip

Archive:  wheel-clean.zip
   creating: wheel-clean/
  inflating: wheel-clean/2017-09-06-White-Supremacist-Symbol-Tarnishes-Old-DeKalb-Courthouse.txt  
  inflating: wheel-clean/2017-09-13-Let-the-Games-Begin-Taylor-Swift-Satirizes-her-Reputation-in-NewSingles.txt  
  inflating: wheel-clean/2014-11-11-Crime-Report-11-11-14.txt  
  inflating: wheel-clean/2017-10-18-Doolino-Knows-Best-Falling-Apart.txt  
  inflating: wheel-clean/2016-11-23-Former-U-S-Poet-Laureate-to-Leave-Emory-for-Northwestern.txt  
  inflating: wheel-clean/2019-09-14-DeKalb-County-Issues-Boil-Water-Advisory.txt  
  inflating: wheel-clean/2015-10-24--Bob-s-Burgers-Going-Steady-Into-Season-Six.txt  
  inflating: wheel-clean/2016-03-21-The-Best-Batman-Film-is-One-You-Have-Probably-Never-Seen.txt  
  inflating: wheel-clean/2018-10-10-Great-Times-Await-at-Bad-Times-at-the-El-Royale-.txt  
  inflating: wheel-clean/2015-10-01-College-Is-Worth-More-Than-Advertised.txt  
  inflating: wheel-clean/2017-09-06-Emory-Doctors-Aid-Refu

In [3]:
# import this library for directory/file manipulation
import os

# set the base directory -- note that this may need to change if you've saved a copy
# of this notebook elsewhere 
base_dir = "wheel-clean/"

# read in a list of all the filenames 
docs = os.listdir(base_dir)

# a list for storing the text of all the docs
wheel_docs = []

# iterate through each of the docs in the directory
for doc in docs:
    with open(base_dir + doc, "r") as file:     # open the doc file 
        text = file.read()                      # read the contents of the file 
        wheel_docs.append(text)                   # append the contents of the file to our
                                                # all_docs list for future manipulation

# just take a look at the first item to be sure it worked
print("Filename: " + str(docs[0]) + "\n") 
print(wheel_docs[0])

Filename: 2017-09-20-Fantastic-Bars-and-Where-to-Find-Them-Atlanta-s-Secret-Bars.txt

You've been to Maggie's Neighborhood Bar and Grill — like, way too many times. Maybe you are ready to try something new. Or maybe you haven't been to Maggie's because it's "too mainstream." Yourfavorite movie is "Citizen Kane" and you relish in the superiority of your unique, refined tastes. Or maybe you just love knowing something that other people don't, and your expertise in secrecy makes Varysfrom "Game of Thrones" look like an amateur. If any of these profiles sound familiar, secret bar-hoppingmay be for you. Atlanta is bursting with Prohibition-style bars just waiting to be explored. All you'll need to find them is a spirit for discovery (and an occasional password).Brigantine Beer Parlor: Resting cosily inside Argosy restaurant in the heart of East Atlanta, BrigantineBeer Parlor is the kind of bar that television shows like "How I Met Your Mother" try to recreate.Complete with skee ball, shuffl

## To the TF-IDF calculation... or not (yet)!

Like many other methods, we can calculate tf-idf with just a few lines of code. But because the calculation relies on a process that is, in itself, a pre-processing step for many future methods, and is in itself a little heady, we're going to spend some time loooking under the hood.

So without further ado, introduing.... the `CountVectorizer`!

## But wait... the count what? 

Thus far, we've examined words in terms of their grammar and syntax. We've also looked at words in terms of various units: the word, the line, the document, etc. And we've counted the numbers of each of these. 

From here on out, however, we'll be taking a different approach. We'll be turning the words themselves into numbers, and then applying statistical measures and models to the numbers that represent the words. TF-IDF, as well as machine learning techniques including topic modeling, BERT, similarity, classification, and clustering--essentially, the set of methods we'll be learning in the second half of the course--all rely on this basic transformation.

## Introducing scikit-learn! 

We've now reached another milestone--our first use of scikit-learn (often abbreviated as sk-learn), Python's major machine learning library, which also happens to be crucial to many of the more advanced methods named above. 

We're going to use this to transform our words into numbers.

We'll begin by importing sk-learn's `CountVectorizer`, which [converts a collection of text documents into a matrix of token counts](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). 

I think I've used the word "token" in passing before, but here I'll take a minute to formally define it, along with some related terms.

## Tokens, features, document-term matrices

According to the [Stanford NLP group](https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html), a *token* is "an instance of a sequence of characters in some particular document that are grouped together as a useful semantic unit for processing." The unit of the token is usually the word, but it can also be the sentence, the subword, or anything else that makes sense for that particular task.

Take this famous phrase, for example:

"To be or not to be"

This line has six tokens: "to", "be", "or", "not", "to", "be".

It has four features: "to", "be", "or", "not"

But wait, what is a feature?

In this case, a *feature* is a unique token in the corpus. (Caveat: features, like tokens, can actually be anything that makes sense for the task, but for the purposes of turning words into numbers, features are most often unique words, or "terms," as they're also sometimes called).

When sk-learn's CountVectorizer does its thing, it first *tokenizes* all of the documents in the corpus--that is, it breaks up each document into its individual tokens--and then then creates a *document-term matrix* that counts up how many times each term, or feature, appears in each document. 

For example, the document-term matrix for the line above might look something like this:

|   | to | be | or | not |
|---|----|----|----|-----|
|   | 2  | 2  | 1  | 1   |


If we add in the second part of that phrase as a new document, we might get something like this:


|         | to | be | or | not | that | is | the | question |
|---------|----|----|----|-----|------|----|-----|----------|
| line 1  | 2  | 2  | 1  | 1   | 0    | 0  | 0   | 0        |
| line 2  | 0  | 0  | 0  | 0   | 1    | 1  | 0   | 0        |


But enough of vectorizing by hand; let's try it out using sk-learn!


## Importing sk-learn's CountVectorizer




In [4]:
# import CountVectorizer from sk-learn
from sklearn.feature_extraction.text import CountVectorizer

## Vectorize a teeny corpus

Now let's vectorize a teeny corpus we can see:

In [5]:
# here's our corpus: the first stanza of "Persimmons" in which each line is its own document
corpus = [
    'In sixth grade Mrs. Walker',
    'slapped the back of my head',
    'and made me stand in the corner',
    'for not knowing the difference',
    'between persimmon and precision.',
    'How to choose',
]

# instantiate the CountVectorizer object
# note that this is the same conceptual process we used to instantiate
# the VADER sentiment analysis object, and the spaCy document object
cv=CountVectorizer()

# this steps generates document-term matrix for the doc; 
# it's required before you do almost anything else
dtm=cv.fit_transform(corpus)

# this method gives us the feature names that the CountVectorizer vectorized:
features = cv.get_feature_names_out()

# this method turns our doc-term matrix into an array that can be manipulated:
dtm_array = dtm.toarray()

print("All of the features in our corpus:")
print(str(features))

print ("\nAnd their counts in each of the \"documents,\" each of which is really just a single line of the poem:")
print(dtm_array)

All of the features in our corpus:
['and' 'back' 'between' 'choose' 'corner' 'difference' 'for' 'grade'
 'head' 'how' 'in' 'knowing' 'made' 'me' 'mrs' 'my' 'not' 'of' 'persimmon'
 'precision' 'sixth' 'slapped' 'stand' 'the' 'to' 'walker']

And their counts in each of the "documents," each of which is really just a single line of the poem:
[[0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0]
 [1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]]


In [6]:
# here is some code that uses dataframes to make the above slightly more legible
# note that this is now the second or third time I've mentioned Python dataframes
# and said we'll talk about them later--we will, promise!

import pandas as pd

df = pd.DataFrame(data=dtm_array,columns=features)
print(df)

   and  back  between  choose  corner  difference  for  grade  head  how  ...  \
0    0     0        0       0       0           0    0      1     0    0  ...   
1    0     1        0       0       0           0    0      0     1    0  ...   
2    1     0        0       0       1           0    0      0     0    0  ...   
3    0     0        0       0       0           1    1      0     0    0  ...   
4    1     0        1       0       0           0    0      0     0    0  ...   
5    0     0        0       1       0           0    0      0     0    1  ...   

   not  of  persimmon  precision  sixth  slapped  stand  the  to  walker  
0    0   0          0          0      1        0      0    0   0       1  
1    0   1          0          0      0        1      0    1   0       0  
2    0   0          0          0      0        0      1    1   0       0  
3    1   0          0          0      0        0      0    1   0       0  
4    0   0          1          1      0        0      0  

Let's take a minute to figure out what we're looking at:

* Each column is a feature, or "term," labeled with the name of the term, which in this case is a unique token
* Each row is a document, labeled in order of being ingested
* The "1" in row 0 of the "grade" column means that the term "grade" appears 1 time in the first document... and so on.  

## Vectorizing a corpus from a set of files

The reality is that you almost always will be vectorizing a corpus from a set of files, and not a list that you type in by hand. This is how you'd do it with the Emory Wheel dataset that we loaded earlier:

In [7]:
# instantiate the vectorizer, as before
cv=CountVectorizer()

# generates document-term matrix for all the docs
dtm=cv.fit_transform(wheel_docs)

# get the feature names aka terms
features = cv.get_feature_names_out()

# take a look at the first 25 features
print(features[0:24])

['00' '000' '00000' '000academic' '000amounts' '000and' '000annually'
 '000attendees' '000award' '000bond' '000cases' '000doses' '000employees'
 '000followers' '000from' '000in' '000m' '000meter' '000mrun'
 '000msteeplechase' '000of' '000others' '000palestinians' '000people']


In [8]:
# you can also check the overall shape of the doc-term matrix 
dtm.shape

(4008, 100303)

**What does this tell you about how many documents there are? What about the number of features?**

## Helpful CountVectorizer Parameters

Since we're on the subject of the `CountVectorizer`, here are a few more helpful `CountVectorizer` parameters to know about:



In [9]:
# lowercase all words -- this is True by default, but if you want to preserve case,
# you can set lowercase to False
cv_caps = CountVectorizer(lowercase=False)

# generates document-term matrix for all the docs
dtm2=cv_caps.fit_transform(wheel_docs)

# check the shape
dtm2.shape

(4008, 111808)

So, there are more terms since it's not merging the uppercase and the lowercase versions of each word together.

Another parameter to know about has to do with stopwords. These are common words like "and", "not", "or", etc. that are not usually that interesting.

In [10]:
# use the built-in English stopwords list
cv_no_stops = CountVectorizer(stop_words='english')

# generates document-term matrix for all the docs
dtm3=cv_no_stops.fit_transform(wheel_docs)

# check the shape
dtm3.shape

(4008, 99999)

In [11]:
# use a custom stopwrods list
cv_no_stops = CountVectorizer(stop_words=['emory','student','atlanta'])

# generates document-term matrix for all the docs
dtm4=cv_no_stops.fit_transform(wheel_docs)

# check the shape
dtm4.shape

(4008, 100300)

One last helpful feature of CountVectorizer is that you can tell it very easily to tokenize by ngrams as well as words. To wit:

In [12]:
bigram_cv = CountVectorizer(analyzer='word', ngram_range=(2, 2))

# generates document-term matrix for all the docs
dtm5=bigram_cv.fit_transform(wheel_docs)

# get the feature names -- bigrams in this case
features = bigram_cv.get_feature_names_out()

# take a look at the first 25 features
print(features[0:24])

['00 00' '00 15' '00 23' '00 27' '00 28' '00 37' '00 43' '00 63' '00 64'
 '00 75' '00 87' '00 94' '00 agent' '00 an' '00 and' '00 as' '00 at'
 '00 bike' '00 but' '00 closely' '00 earning' '00 emory' '00 epd' '00 for']


## Converting the doc-term matrix to a dictionary

Finally, here is some helpful code for creating a dictonary with the features as keys and the counts as values. Don't worry about being able to parse all the syntax unless you feel like it.

In [ ]:
# here's our number of features
num_feats = dtm.shape[1]

# here's a dictionary to store the features and counts key/value pairs
feature_dict = {}

for x in range(num_feats):      # the for x in range() syntax is how you iterate over integers
    key = cv.get_feature_names_out()[x]  # this gets the feature name at position [x]
    value = dtm.toarray().sum(axis=0)[x]  # this sums the counts of the feature at
                                          # position [x] for all documents
    
    feature_dict[key] = value # add the new key/value pair to the dictionary
    
# then sort the dictionary in order of counts
sortFeats = sorted(feature_dict.items(), key=lambda x: x[1], reverse=True)

# for more on the sorted function, see: https://www.w3schools.com/python/ref_func_sorted.asp
# for more on lambda functions, see: https://towardsdatascience.com/lambda-functions-with-practical-examples-in-python-45934f3653a8

# then print top 30

for item in sortFeats[0:30]:
    print(str(item[0]) + ": " + str(item[1]))

## Return to TF-IDF

At long last, we're ready to calculate the TF-IDF scores for our corpus!

We're going to do this step by step at first, to make sure that you understand each of the processes, and then at the end, you'll see how to do this in only a few lines of code.

In [ ]:
# import the TF-IDF libraries
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

## Create document-term matrix

We'll use a doc-term matrix to calculate tf-idf. We already know how to do this step

In [ ]:
#instantiate CountVectorizer()
cv=CountVectorizer(stop_words='english') # using stopwords this time

# this steps generates document-term matrix for the docs
dtm=cv.fit_transform(wheel_docs)

# check shape
dtm.shape

**How many articles do we have in this document-term matrix and how many unique features/terms?**

That line, as we learned last clas, tells us that we have 4008 rows, one for each document in the corpus, and 99,999 columns, one for each word (minus single character words, which the tokenizer excludes, as well as the default stopwords, which we've indicated should be excluded with the stop_words='english' parameter above).

We can also look at the whole vocabulary like this:

In [ ]:
cv.vocabulary_

The numbers above are the indices for each feature, not the word counts. But we can use the indicies in order to generate our word counts. 

Note that we did a similar thing above using `cv.get_feature_names_out()`. This is arguably more efficient.

In [ ]:
sum_words = dtm.sum(axis=0) # sum_words is a vector that contains the number of times each word appears in all 
                            # the docs in the corpus. In other words, we are summing the elements for each column 
                            # of the doc-term matrix and storing those counts as a vector 

# then sort the list of tuples that contain the word and their occurrence in the corpus.
# tuples are Python's name for single variables that actually store multiple variables, 
# like the word and index in the vocabulary attribute above 

words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()] # rememeber list comprehension! 

words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

# display the top 10
words_freq[:10]

We can already see some words with the most counts don't seem too distinctive: "emory" and "students," for example. It's not surprising that those are the most frequently occurring words since the Wheel is a newspaper about Emory students.

So now let's calculate the IDF values so that we can balance them out. While we could also calculate these by hand, sk-learn makes it really easy to do it in a few lines of code, so we'll use that instead. 

## Initialize TfidfTransformer

When you initialize TfidfTransformer, you can choose to set it with different parameters. These parameters will change the way you calculate tf–idf. The recommended way to run `TfidfTransformer` is with smoothing (`smooth_idf = True`) and normalization (`norm='l2'`) turned on. These parameters will better account for differences in document length, and, overall, they'll produce more meaningful tf–idf scores. 

In [ ]:
# Call tfidf_transformer.fit on the word count vector we computed earlier.
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(dtm)

## Print inverse document frequence (idf) values

In [ ]:
# make a dataframe for the idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

In the table above, the words at the top are those that appear in the most number of documents, across all of the corpus; and the words at the bottom are those that appear in the least number of documents.

Once again, it makes sense that words like "Emory" and "said" are at the top. It's a newspaper, after all! 

The words at the bottom appear to be either typos or whitespace errors. I'm guessing most of those appear only once across the entire corpus. 

## IDF by the numbers

But what are these numbers that we're looking at?

The most direct formula would be **N/df<sub>i</sub>**, where N represents the total number of documents in the corpus, and df is the number of documents in which the term appears. 

However, many implementations of tf-idf, including scikit-learn, which we are using, normalize the results with additional operations. 

In tf-idf, normalization is generally used in two ways, and for two reasons: first, to prevent bias in term frequency from terms in shorter or longer documents; and second, as above, to calculate each term’s idf value. 

Scikit-learn’s implementation of tf-idf represents N as **N+1**, calculates the natural logarithm of **(N+1)/df<sub>i</sub>**, and then adds **1** to the final result. Here is this same thing formatted slightly more nicely:

<img src="http://lklein.com/wp-content/uploads/2019/10/Screen-Shot-2019-10-02-at-11.52.31-PM.png">

**Important note!** This is only one way to calculate TF-IDF. There are many, many versions. The number itself isn't important. It's the *ranking* that the number enables that's most interesting to us. Because one you have the IDF values, you can now compute the tf-idf scores for any document or set of documents. 

So now let’s compute tf-idf scores for the documents in our corpus.


## Produce & print tf-idf scores

Once you have the idf values, you can compute the tf-idf scores for any document or set of documents. Let’s compute tf-idf scores for the documents in our corpus.

In [ ]:
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(dtm)

Now, let’s print the tf-idf values of the first document to see if they make sense. 

We'll place the tf-idf scores from the first document into a pandas dataframe and sort the dataframe in descending order of scores.

In [ ]:
feature_names = cv.get_feature_names_out()

#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores for the first doc
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

Notice that only certain words have scores. This is because only the words in this document have a tf-idf score and everything else, from other documents, shows up as zeroes.

## tf-idf: the fast way

Since we're now tf-idf pros, we're going to use scikit-learn's all-in-one tf-idf vectorizer to do this entire notebook again in two lines of code. 

In [ ]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english', use_idf=True) # excludings stopwords again
 
# send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(wheel_docs)

In [ ]:
# place tf-idf values for all docs in a pandas dataframe
# tfidf_df = pd.DataFrame(tfidf_vectorizer_vectors.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names())

tfidf_df = pd.DataFrame(tfidf_vectorizer_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names())

tfidf_df

In [ ]:
# Add row for number of times word appears in all documents
tfidf_df.loc['Document Frequency'] = (tfidf_df > 0).sum()

## Let's explore!

We can look at specific words and how they appear in our newspaper corpus. I've entered five words below that we might want to investigate:

In [ ]:
tfidf_slice = tfidf_df[['dooley', 'fall', 'pumpkin', 'spice', 'break']]
tfidf_slice

**Does this output make sense? What does it tell you about which articles you might want to go read? What about some research questions you might ask of this corpus using TF-IDF?**

## Cosine similarity

Just so you know how to do everything in that Pudding Hip-Hop feature, this is how you calculate cosine similiarty between documents on the basis of their tf/idf scores:

In [ ]:
# CALCULATE SIMILARITY TO FIRST DOC 

from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidf_vectorizer_vectors[0:1], tfidf_vectorizer_vectors)

*Lauren F. Klein wrote version 1.0 of this notebook in 2019 based of tutorials by [Matthew Lavin](https://programminghistorian.org/en/lessons/analyzing-documents-with-tfidf) and [Kavita Ganesan](https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.XZVlcOdKhSw). Dan Sinykin supplemented it with material from Melanie Walsh's chapter [TF-IDF](https://melaniewalsh.github.io/Intro-Cultural-Analytics/features/Text-Analysis/TF-IDF.html) in 2020. Lauren Klein updated it again in 2021 and 2022.*

